In [1]:
import cv2
import glob
import json as JSON
import os
import numpy as np

In [2]:
pwd

'/workspace/storage'

---
## train raw image crop

In [10]:
file_path = './mixed/train/label/'
file_path_list = glob.glob(file_path+'*.png')

In [11]:
len(file_path_list)

324

In [36]:
os.makedirs('./roads/crop/val/label')

In [12]:
save_directory = './mixed/crop/train/label/'

In [6]:
from tqdm import tqdm

def crop_and_save(save_directory, image_path, crop_size):
    
    cnt = 1
    image = cv2.imread(image_path)
    img_name = image_path[image_path.rfind('/')+1:-4]
    
    for i in range(image.shape[0] // crop_size):
        for j in range(image.shape[1] // crop_size):
            x, y = j*crop_size, i*crop_size
            h, w = crop_size, crop_size
            # cropped_images.append(image[y:y+h, x:x+w])
            cropped = image[y:y+h, x:x+w]
            f_name = f'{img_name}_{cnt:02d}.png'
            cnt += 1
            cv2.imwrite(save_directory+f_name, cropped)

In [13]:
for file in tqdm(file_path_list):
    crop_and_save(save_directory, file, 256)

100%|██████████| 324/324 [00:09<00:00, 35.85it/s]


In [14]:
save_directory

'./mixed/crop/train/label/'

In [15]:
check = glob.glob(save_directory+'*.png')
len(check) / 16

324.0

---
## train label image crop

In [24]:
json_path = glob.glob('./building/train/label/*.json')

In [26]:
def parse_json(json_file_path):
    
    with open(json_file_path) as f:
        json_file = JSON.load(f)
        polygon_list = []
        file_name_list = []
        for feature in json_file['features']:
            if feature['properties']['building_imcoords']:
                polygon = list(map(float, feature['properties']['building_imcoords'].split(',')))
                polygon = np.array(polygon, np.int32).reshape(-1, 1, 2)
                polygon_list.append(polygon)
                file_name_list.append(feature['properties']['image_id'][:-4])
    
    return polygon_list, set(file_name_list)

In [27]:
def draw_label_then_crop_and_save(directory, polygon, img_name, crop_size):

        image = np.zeros((1024, 1024, 3), np.uint8)
        image = cv2.fillPoly(image, polygon_list, (0,255,0))
        image = cv2.polylines(image, polygon_list, True, (255, 255, 255), 3)
        image = cv2.fillPoly(image, polygon_list, (0,255,0))
        
        cnt = 1
        for i in range(image.shape[0] // crop_size):
            for j in range(image.shape[1] // crop_size):
                x, y = j*crop_size, i*crop_size
                h, w = crop_size, crop_size
                # cropped_images.append(image[y:y+h, x:x+w])
                cropped = image[y:y+h, x:x+w]
                f_name = f'{str(img_name)[2:-2]}_{cnt:02d}.png'
                cnt += 1
                cv2.imwrite(directory+f_name, cropped)

In [28]:
os.makedirs('./building/crop/train/label')

In [29]:
directory = './building/crop/train/label/'
for json_file in json_path:
    polygon_list, file_name = parse_json(json_file)
    draw_label_then_crop_and_save(directory, polygon_list, file_name, 256)

In [31]:
json_list = glob.glob(directory+'*.png')

In [33]:
len(json_list)

19808

---
## validatain raw image crop

In [34]:
os.makedirs('./building/crop/val/raw')

In [35]:
file_path = './building/val/raw/'
file_path_list = glob.glob(file_path+'*.png')

In [36]:
save_directory = './building/crop/val/raw/'

In [38]:
for file in file_path_list:
    crop_and_save(save_directory, file, 256)

In [37]:
len(file_path_list)*16

2544

In [39]:
crop_list = glob.glob(save_directory+'*.png')
len(crop_list)

2544

---
## validation label image crop

In [40]:
os.makedirs('./building/crop/val/label')

In [41]:
json_path = glob.glob('./building/val/label/*.json')

In [42]:
len(json_path)

159

In [43]:
directory = './building/crop/val/label/'
for json_file in json_path:
    polygon_list, file_name = parse_json(json_file)
    draw_label_then_crop_and_save(directory, polygon_list, file_name, 256)

In [44]:
crop_list = glob.glob(directory+'*.png')
len(crop_list)/16

159.0